In [0]:
%tensorflow_version 2.x

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1. Load the dataset 


a. Tip: As the dataset is large, use fewer rows. Check what is working well on your machine and decide accordingly.

In [0]:
project_path = "/content/drive/My Drive/AIML/Colab_Notebook/Residency8/Project/Statistical_NLP/blog-authorship-corpus.zip"

In [0]:
from zipfile import ZipFile
with ZipFile(project_path, 'r') as zip:
  zip.extractall()

In [6]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import re, string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
blogs = pd.read_csv('/content/blogtext.csv')
blogs.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [8]:
blogs.shape

(681284, 7)

In [9]:
blogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      681284 non-null  int64 
 1   gender  681284 non-null  object
 2   age     681284 non-null  int64 
 3   topic   681284 non-null  object
 4   sign    681284 non-null  object
 5   date    681284 non-null  object
 6   text    681284 non-null  object
dtypes: int64(2), object(5)
memory usage: 36.4+ MB


In [0]:
blogs.topic = blogs.topic.astype(str)

In [11]:
blogs.topic.value_counts()

indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufacturing 

In [0]:
# As the dataset is large, picking few topics only
blogs_student = blogs[ (blogs.topic == 'Student') ] 
blogs_tech =   blogs[ (blogs.topic == 'Technology') ]
blogs_edu     = blogs[(blogs.topic == 'Education')]       
blogs_comm_med = blogs[(blogs.topic == 'Communications-Media')]
blogs_int = blogs[(blogs.topic == 'Internet')]
blogs_non_profit = blogs[(blogs.topic == 'Non-Profit')]
blogs_engg = blogs[(blogs.topic == 'Engineering')]
blogs_law = blogs[blogs.topic == 'Law']
blogs_pub = blogs[blogs.topic == 'Publishing']
blogs_sci = blogs[blogs.topic == 'Science']
blogs_gov = blogs[(blogs.topic == 'Government')]
blogs_rel = blogs[(blogs.topic == 'Religion')]
blogs_fashion = blogs[(blogs.topic == 'Fashion')]
blogs_consult = blogs[(blogs.topic == 'Consulting')]

In [0]:
# resampling to reduce the data for each topic
from sklearn.utils import resample

In [0]:
# Replace the data of different topics and have equal number of samples. 

blogs_student = resample(blogs_student, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_tech = resample(blogs_tech, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_edu= resample(blogs_edu, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_comm_med = resample(blogs_comm_med,
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_int = resample(blogs_int, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_non_profit = resample(blogs_non_profit, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_engg = resample(blogs_engg, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_law = resample(blogs_law, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_pub = resample(blogs_pub, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_sci = resample(blogs_sci, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_gov = resample(blogs_gov, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_rel = resample(blogs_rel, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

blogs_consult = resample(blogs_consult, 
                      replace=True,    
                      n_samples=5000,     
                      random_state=108)

In [0]:
# Combine all the data with equal number of samples
df_blogs = pd.concat([blogs_student, blogs_tech, blogs_edu, blogs_comm_med, blogs_int, blogs_non_profit, blogs_engg, blogs_law, blogs_pub, blogs_sci, blogs_gov, blogs_rel, blogs_fashion, blogs_consult])

In [16]:
# Updated data 
df_blogs['topic'].value_counts()

Technology              5000
Law                     5000
Religion                5000
Publishing              5000
Non-Profit              5000
Science                 5000
Education               5000
Government              5000
Student                 5000
Communications-Media    5000
Engineering             5000
Consulting              5000
Internet                5000
Fashion                 4851
Name: topic, dtype: int64

In [17]:
df_blogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69851 entries, 577097 to 141834
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      69851 non-null  int64 
 1   gender  69851 non-null  object
 2   age     69851 non-null  int64 
 3   topic   69851 non-null  object
 4   sign    69851 non-null  object
 5   date    69851 non-null  object
 6   text    69851 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.3+ MB


In [18]:
df_blogs.head()

,id,gender,age,topic,sign,date,text
577097,1704362,male,17,Student,Pisces,"05,August,2003",SCENE: Jim sitting at his comput...
492967,1844606,male,23,Student,Taurus,"02,August,2004","The past two nights, I had some sev..."
215511,3421369,male,14,Student,Cancer,"06,August,2004",This morning wwoke up as usual.. Went t...
230585,3515827,male,24,Student,Pisces,"10,August,2004",I am fascinated by Kitty 's collection...
579072,2117806,male,24,Student,Aries,"17,July,2004","'She'll Take What You Give Her, ..."


# 2. Preprocess rows of the “text” column 


a. Remove unwanted characters

b. Convert text to lowercase

c. Remove unwanted spaces

d. Remove stopwords



In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
def preprocess_text(strg):
  try:    
    strg = re.sub(r'^https?:\/\/<>.*[\r\n]*', '', strg, flags=re.MULTILINE)
    strg = re.sub(r"[^A-Za-z]", " ", strg)         
    wrds = strg.strip().lower().split()    
    wrds = [w for w in wrds if not w in stop_words]
    return " ".join(wrds)	
  except:
    return ""

In [0]:
df_blogs['text'] = df_blogs['text'].apply(lambda x: preprocess_text(x))

In [22]:
df_blogs['text'].head(10)

577097    scene jim sitting computer slouched chair play...
492967    past two nights severly disturbing dreams tv n...
215511    morning wwoke usual went school sit quadrangle...
230585    fascinated kitty collection found letters pict...
579072    take give long free nbsp progress note thesis ...
496079    uniquely fortunate person know led fairly char...
464263    yesterday chatted almost day tried write pages...
452532    pc forum leave computer dont know dont introdu...
411103    one run truth forever matter fast run catch en...
143607    laundry bag hand waiting elevator arrive go ba...
Name: text, dtype: object

## 3. As we want to make this into a multi-label classification problem, you are required to merge
## all the label columns together, so that we have all the labels together for a particular sentence

a. Label columns to merge: “gender”, “age”, “topic”, “sign”

b. After completing the previous step, there should be only two columns in your dataframe i.e. “text” and “labels” as shown in the below image


In [0]:
cols = ['gender','age','topic','sign']
df_blogs['labels'] = df_blogs[cols].apply(lambda row: ','.join(row.values.astype(str)), axis=1)

In [24]:
df_blogs.head()

,id,gender,age,topic,sign,date,text,labels
577097,1704362,male,17,Student,Pisces,"05,August,2003",scene jim sitting computer slouched chair play...,"male,17,Student,Pisces"
492967,1844606,male,23,Student,Taurus,"02,August,2004",past two nights severly disturbing dreams tv n...,"male,23,Student,Taurus"
215511,3421369,male,14,Student,Cancer,"06,August,2004",morning wwoke usual went school sit quadrangle...,"male,14,Student,Cancer"
230585,3515827,male,24,Student,Pisces,"10,August,2004",fascinated kitty collection found letters pict...,"male,24,Student,Pisces"
579072,2117806,male,24,Student,Aries,"17,July,2004",take give long free nbsp progress note thesis ...,"male,24,Student,Aries"


In [0]:
updated_blogs = df_blogs[['text','labels']]

In [0]:
# convert all the words in column labels to lower case. 
updated_blogs['labels'] = updated_blogs['labels'].str.lower()

In [27]:
updated_blogs.head()

,text,labels
577097,scene jim sitting computer slouched chair play...,"male,17,student,pisces"
492967,past two nights severly disturbing dreams tv n...,"male,23,student,taurus"
215511,morning wwoke usual went school sit quadrangle...,"male,14,student,cancer"
230585,fascinated kitty collection found letters pict...,"male,24,student,pisces"
579072,take give long free nbsp progress note thesis ...,"male,24,student,aries"


## 4. Separate features and labels, and split the data into training and testing 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(updated_blogs.text, updated_blogs.labels, random_state=108, test_size=0.3)

In [29]:
#Traing data
print(X_train.shape)
print(y_train.shape)

(48895,)
(48895,)


In [30]:
#Test Data
print(X_test.shape)
print(y_test.shape)

(20956,)
(20956,)


# 5. Vectorize the features 
a. Create a Bag of Words using count vectorizer

i. Use ngram_range=(1, 2)

ii. Vectorize training and testing features


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
count_vect = CountVectorizer(ngram_range=(1,2))

In [33]:
count_vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [34]:
len(count_vect.vocabulary_)

2768496

b. Print the term-document matrix

In [0]:
X_train_ct = count_vect.transform(X_train)

In [36]:
X_train_ct.shape

(48895, 2768496)

In [37]:
#sparse matrix - stores index with Non-zero values as most of the values are zero they are not stored. 
X_train_ct[0]

<1x2768496 sparse matrix of type '<class 'numpy.int64'>'
	with 158 stored elements in Compressed Sparse Row format>

In [38]:
# In sparse matrix only the index values are 1 and others are 0 (sparese matrix- to save memory)
print(X_train_ct[0])

  (0, 4168)	1
  (0, 4171)	1
  (0, 83553)	1
  (0, 83579)	1
  (0, 170298)	1
  (0, 174769)	1
  (0, 239554)	2
  (0, 239989)	1
  (0, 240225)	1
  (0, 240282)	2
  (0, 240348)	1
  (0, 240349)	1
  (0, 265351)	1
  (0, 265382)	1
  (0, 274088)	1
  (0, 274280)	1
  (0, 299867)	1
  (0, 300451)	1
  (0, 305050)	2
  (0, 305089)	1
  (0, 305122)	1
  (0, 366832)	1
  (0, 366906)	1
  (0, 407962)	1
  (0, 505493)	1
  :	:
  (0, 2528357)	1
  (0, 2557916)	1
  (0, 2557928)	1
  (0, 2563027)	2
  (0, 2564297)	1
  (0, 2566892)	1
  (0, 2578328)	1
  (0, 2578467)	1
  (0, 2619473)	1
  (0, 2619684)	1
  (0, 2684705)	1
  (0, 2685449)	1
  (0, 2722912)	1
  (0, 2723108)	1
  (0, 2732559)	1
  (0, 2732581)	1
  (0, 2738328)	1
  (0, 2738339)	1
  (0, 2744819)	1
  (0, 2745092)	1
  (0, 2748057)	2
  (0, 2748323)	1
  (0, 2749213)	1
  (0, 2750579)	1
  (0, 2750859)	1


In [0]:
X_test_ct = count_vect.transform(X_test)

In [62]:
X_test_ct.shape

(20956, 2768496)

In [63]:
X_test_ct[0]

<1x2768496 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [64]:
print(X_test_ct[0])

  (0, 220375)	1
  (0, 851011)	1
  (0, 1113494)	1
  (0, 2191720)	1
  (0, 2616653)	1


## 6. Create a dictionary to get the count of every label 
i.e. the key will be label name and value will
be the total count of the label. Check below image for reference 

In [0]:
labelVector = CountVectorizer()

In [0]:
lbl_ct = labelVector.fit_transform(updated_blogs['labels'])

In [41]:
print("No. of Unique Labels : ", len(labelVector.vocabulary_))

No. of Unique Labels :  56


In [42]:
labelVector.get_feature_names()

['13',
 '14',
 '15',
 '16',
 '17',
 '23',
 '24',
 '25',
 '26',
 '27',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 'aquarius',
 'aries',
 'cancer',
 'capricorn',
 'communications',
 'consulting',
 'education',
 'engineering',
 'fashion',
 'female',
 'gemini',
 'government',
 'internet',
 'law',
 'leo',
 'libra',
 'male',
 'media',
 'non',
 'pisces',
 'profit',
 'publishing',
 'religion',
 'sagittarius',
 'science',
 'scorpio',
 'student',
 'taurus',
 'technology',
 'virgo']

In [0]:
lbl_dict = labelVector.vocabulary_

In [44]:
lbl_dict

{'13': 0,
 '14': 1,
 '15': 2,
 '16': 3,
 '17': 4,
 '23': 5,
 '24': 6,
 '25': 7,
 '26': 8,
 '27': 9,
 '33': 10,
 '34': 11,
 '35': 12,
 '36': 13,
 '37': 14,
 '38': 15,
 '39': 16,
 '40': 17,
 '41': 18,
 '42': 19,
 '43': 20,
 '44': 21,
 '45': 22,
 '46': 23,
 '47': 24,
 '48': 25,
 'aquarius': 26,
 'aries': 27,
 'cancer': 28,
 'capricorn': 29,
 'communications': 30,
 'consulting': 31,
 'education': 32,
 'engineering': 33,
 'fashion': 34,
 'female': 35,
 'gemini': 36,
 'government': 37,
 'internet': 38,
 'law': 39,
 'leo': 40,
 'libra': 41,
 'male': 42,
 'media': 43,
 'non': 44,
 'pisces': 45,
 'profit': 46,
 'publishing': 47,
 'religion': 48,
 'sagittarius': 49,
 'science': 50,
 'scorpio': 51,
 'student': 52,
 'taurus': 53,
 'technology': 54,
 'virgo': 55}

## 7. Transform the labels - 
As we have noticed before, in this task each example can have multiple tags. To deal with
such kind of prediction, we need to transform labels in a binary form and the prediction will be
a mask of 0s and 1s. For this purpose, it is convenient to use MultiLabelBinarizer from sklearn

a. Convert your train and test labels using MultiLabelBinarizer

In [45]:
print(y_train[0:1])
print(y_test[0:1])

550015    male,35,religion,aquarius
Name: labels, dtype: object
1936    male,35,technology,aries
Name: labels, dtype: object


In [0]:
y_train = [set(i.split(',')) for i in y_train]
y_test = [set(i.split(',')) for i in y_test]

In [47]:
print(y_train[0:1])
print(y_test[0:1])

[{'aquarius', 'religion', '35', 'male'}]
[{'technology', 'aries', '35', 'male'}]


In [48]:
y_labels = [set(i.split(',')) for i in updated_blogs.labels]
print(y_labels[0])

{'17', 'pisces', 'student', 'male'}


In [49]:
len(y_labels[0])

4

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
MB = MultiLabelBinarizer(classes=labelVector.get_feature_names())

In [0]:
y_train = MB.fit_transform(y_train)
y_test = MB.fit_transform(y_test)

In [53]:
print("Train : ", len(y_train[0]))
print("Test : ", len(y_test[0]))

Train :  56
Test :  56


In [54]:
len(MB.classes_)

56

In [0]:
y_labels_MB = MB.transform(y_labels)

In [56]:
len(y_labels_MB[0])

56

# 8. Choose a classifier - 
In this task, we suggest using the One-vs-Rest approach, which is implemented in
OneVsRestClassifier class. In this approach k classifiers (= number of tags) are trained. As a
basic classifier, use LogisticRegression. It is one of the simplest methods, but often it
performs good enough in text classification tasks. It might take some time because the
number of classifiers to train is large.

a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on
every label

b. As One-vs-Rest approach might not have been discussed in the sessions, we are
providing you the code for that

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
lr = LogisticRegression(solver='lbfgs')
orc = OneVsRestClassifier(lr)

In [59]:
orc.fit(X_train_ct, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred = orc.predict(X_test_ct)

# 9. Fit the classifier, make predictions and get the accuracy
a. Print the following

i. Accuracy score

ii. F1 score

iii. Average precision score

iv. Average recall score

v. Tip: Make sure you are familiar with all of them. How would you expect the
things to work for the multi-label scenario? Read about micro/macro/weighted
averaging

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

from functools import partial

In [0]:
def scores(y_true, y_pred, type='all'):
  
    scores = []
    avg_method = ['macro', 'micro', 'weighted']
    
    for avg in avg_method: 
        if type in ['all','f1']:
            f1_sc = partial(f1_score, average=avg)
            print(" F1 Score("+avg+") : ", f1_sc(y_true, y_pred))
                
        if type in ['all','avg']:
            avg_prec_sc = partial(average_precision_score, average=avg)
            print(" Average Precision Score("+avg+") : ", avg_prec_sc(y_true, y_pred))
                
        if type in ['all','recall']:
            recall_sc = partial(recall_score, average=avg)
            print(" Recall Score("+avg+") : ", recall_sc(y_true, y_pred))
            

In [103]:
print("Accuracy Score:" , accuracy_score(y_test, y_pred))

Accuracy Score: 0.26402939492269517


In [104]:
print("F1 Score:")
scores(y_test, y_pred, type='f1')

F1 Score:
 F1 Score(macro) :  0.43324563794475474
 F1 Score(micro) :  0.5716479385771984
 F1 Score(weighted) :  0.5421135812115457


In [105]:
print("Average Precission Score:")
scores(y_test, y_pred, type='avg')

Average Precission Score:
 Average Precision Score(macro) :  nan
 Average Precision Score(micro) :  0.3958298969880275
 Average Precision Score(weighted) :  0.4252847107100592


In [106]:
print("Average Recall Score:")
scores(y_test, y_pred, type='recall')

Average Recall Score:
 Recall Score(macro) :  0.31052752840302106
 Recall Score(micro) :  0.44219252660232616
 Recall Score(weighted) :  0.44219252660232616


## 10. Print true label and predicted label for any five examples

In [107]:
print(y_test[0])
print(y_pred[0])

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [0]:
y_pred_invT = MB.inverse_transform(y_pred)
y_test_invT = MB.inverse_transform(y_test)

In [0]:
def trueLB_vs_predLB(n):
    for i in range(1,n):
        print("\033[91m" + "Actual", i, ":"+"\033[0m", y_test_invT[i], "\033[91m" +"\tPredicted", i, ":"+"\033[0m", y_pred_invT[i],"\n")

In [102]:
trueLB_vs_predLB(6)

Actual 1 : ('25', 'female', 'taurus') 	Predicted 1 : ('25', 'female', 'taurus') 

Actual 2 : ('44', 'leo', 'male', 'publishing') 	Predicted 2 : ('male',) 

Actual 3 : ('40', 'capricorn', 'consulting', 'male') 	Predicted 3 : ('40', 'capricorn', 'consulting', 'male') 

Actual 4 : ('23', 'male', 'science', 'virgo') 	Predicted 4 : ('23', 'male', 'science', 'virgo') 

Actual 5 : ('45', 'male', 'technology', 'virgo') 	Predicted 5 : ('male', 'technology') 

